<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/main/MVA2024_ex15notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2024 ex15notebookB

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo15.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2024

---
## 機械学習のデモ
---

この授業で学んだ内容の延長上には様々な話がありますが，ここでは，その一つの方向である **機械学習** についてのデモを実行してみましょう．
機械学習については，3年1Q,2Qの「機械学習I/II」で学ぶことができます．

「機械学習I/II」のページ https://www-tlab.math.ryukoku.ac.jp/wiki/?ML


In [1]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

import pickle
import json
import io
import requests
from PIL import Image

# Python の深層学習フレームワーク PyTorch のいろいろ
import torch
from torchvision import models, transforms
from torchvision.io.image import read_image, ImageReadMode
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


---
### 1000種類の物体を識別するニューラルネットを動かしてみよう

ニューラルネットワークを用いた画像識別の実験をやってみましょう． いろんなものが識別できた方が楽しいと思いますので，1000種類の物体を識別させてみます． ただし，そういうことができるニューラルネットワークを一から学習させようとすると大変（↑ののりで実行したら何日もかかります）ですので，ここではすでに学習済みのものを利用することにします．

#### 準備

まずは実験に使う画像データを準備します．

**注意: 以下の画像をこの実験以外の目的で使用してはいけません**

In [ ]:
!wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/animalphoto.pickle
with open('animalphoto.pickle', 'rb') as f:
    hoge = pickle.load(f)

imgList = []
for d in hoge:
    img = Image.frombytes(**d)
    imgList.append(img)
    fig, ax = plt.subplots(1)
    ax.imshow(img)
    ax.axis('off')
    plt.show()

1000個あるクラスの番号とクラスラベルの対応表を作成します．

In [ ]:
# ImageNet クラスラベルを表すJSONファイルを入手
!wget -nc https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json
class_index = json.load(open('imagenet_class_index.json', 'r'))

K = len(class_index)
labels = {}
for ik, key in enumerate(class_index.keys()):
    labels[ik] = f'{class_index[key][0]} {class_index[key][1]}'
    if 276 <= ik < 300:
        print(ik, labels[ik])

上記の出力は，1000クラスのうちの一部のものの名前を表します．「ネコ」みたいなものも1クラスではなく，281番 'tabby'（ぶち柄の猫），282番 'tiger_cat'（トラ縞の猫）等々のように分かれています．

学習済みのニューラルネットワークのパラメータを入手します．
規模の大きいネットワークでパラメータがたくさんありますので，読み込みに少し時間がかかります．

In [ ]:
vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
vgg16.eval()
print(vgg16)

上記のセルを実行すると，次のことが行われます．
1. ニューラルネットワークの学習モデルを作成する．
1. 学習済みのネットワークパラメータの値を読み込んでこのネットワークのパラメータに設定する．
1. ネットワークモデルの構造を表示する．

参考:

ここで用いているのは VGG16 というニューラルネットワークモデルです．イギリスオックスフォード大学の [Visual Geometry Group(VGG)](https://www.robots.ox.ac.uk/~vgg/) という研究グループが作った VGG-net というニューラルネットワークモデルのうちの，層が16層あるものです．
VGG-net は，2014年に行われた [ILSVRC2014](https://www.image-net.org/challenges/LSVRC/2014/) という画像識別のコンペティションで世界第2位となったニューラルネットワークモデルです．
ILSVRC2014では，[ImageNet](https://www.image-net.org/) という大規模な画像データセットの中から選ばれた 1000 種類の物体の画像約120万枚を学習データとしていました．


画像をネットワークに入力できる形式に加工する処理を定義しておきます．


In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

#### 実験1

上記のサンプル画像たちを VGG16 に識別させてみましょう．


In [ ]:
#@markdown 以下で 0 から 3 までの整数を選ぶと4つのサンプル画像の中から一つを選ぶことができます．
i = 0 #@param [0, 1, 2, 3] {type: 'raw'}
img = imgList[i]

# 画像を表示
fig, ax = plt.subplots(1)
ax.imshow(img)
ax.axis('off')
plt.show()

# VGG16 ネットワークに入力して出力を得る
X = torch.unsqueeze(preprocess(imgList[i]), axis=0)
Y = vgg16(X)
Z = torch.nn.functional.softmax(Y, dim=1)
P = Z[0].detach().numpy()

# 出力の値の大きかった方から5つを表示
for i, ik in enumerate(np.argsort(-P)[:5]):
    print(f'rank{i+1}: {P[ik]:.6f} {labels[ik]}')

上記の画像の下の出力は，1000個のクラスの中で，ネットワークの出力の値が大きかったもの上位5位までの，出力の値とクラス名を表します．
出力の値は，そのクラスと判定することの「確信度」と解釈できます．

#### 実験2

自分で用意した画像でも実験してみましょう．

- 上記では動物の例ばかりですが，1000のクラスの中にはそれ以外にも様々なものがあります（人間のクラスはありません）
- 面白い／不思議な結果が得られたら takataka に見せてくれると喜びます



(1) まずは，ウェブ等で適当な画像を探して，この Colab notebook を実行しているコンピュータへ保存しましょう．ファイル名が長かったり日本語を含んでいる場合は，短い名前に変更しておくのがよいです．

(2) 以下のセルをコメントの指示にしたがって修正してから実行して，ファイルをアップロードします．

In [ ]:
# 以下の 0 を 1 に修正してセルを実行しよう
#
if 1 == 0:
    # Colab へファイルをアップロード
    try:
        from google.colab import files
        rv = files.upload()
    except:
        print('このコードは Colab 以外の環境では実行できないよ．')

(3) 以下のセルの1行目のファイル名を上記でアップロードしたものに変えて実行しましょう．アルファベット大文字小文字の区別もありますので注意．うまくいけば画像が表示されるはずです．

In [ ]:
fn = 'hoge.jpg' # ここを拡張子含めてアップロードしたファイルの名前に変更
try:
    myimg = Image.open(fn)
    if myimg.mode != 'RGB':
        myimg = myimg.convert('RGB')
    fig, ax = plt.subplots(1)
    ax.imshow(myimg)
    ax.axis('off')
    plt.show()
except FileNotFoundError:
    print(f'ファイル {fn} が見つかりません．このセルの1行目を正しいファイル名に変更して実行し直してね．')

(4) 以下のセルを実行すれば結果が表示されます．

In [ ]:
# VGG16 ネットワークに入力して出力を得る
X = torch.unsqueeze(preprocess(myimg), axis=0)
Y = vgg16(X)
Z = torch.nn.functional.softmax(Y, dim=1)
P = Z[0].detach().numpy()

# 出力の値の大きかった方から5つを表示
for i, ik in enumerate(np.argsort(-P)[:5]):
    print(f'rank{i+1}: {P[ik]:.6f} {labels[ik]}')

---
### 物体検出してみよう

画像の中に写っている複数の物体を検出する問題を扱うニューラルネットを動かしてみましょう．このニューラルネットは，一枚の画像を入力すると，画像中の物体ごとに，その位置とそれが何であるかを表す情報を出力します．

#### 準備

ここでは，FasterRCNN と呼ばれる，ニューラルネットによる物体検出の仕組みの事前学習済みモデルを使います．
このモデルは，COCO (https://cocodataset.org/) と呼ばれる大規模画像データセットで 81 種類の物体の検出を学習したものです．

- Shaoqing Ren, Kaiming He, Ross Girshick, and Jian Sun, "Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks," NeurIPS 2015, https://arxiv.org/abs/1506.01497
- https://pytorch.org/vision/0.15/models/generated/torchvision.models.detection.fasterrcnn_resnet50_fpn.html

In [ ]:
# FasterRCNN の事前学習済みモデルを入手

weights = models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
preprocess = weights.transforms()
model = models.detection.fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9)
model = model.to(device)
model.eval()
#print(model)

# 物体のクラスの一覧を出力
n = 0
for cn in weights.meta["categories"]:
    if cn != 'N/A':
        print(n, cn, end='\n')
        n += 1

実験用のサンプル画像を入手します．これらは，COCOデータセットの学習用または評価用のデータの一部です．

In [ ]:
# 画像を入手
urlDict = {
    'dog':'https://farm6.staticflickr.com/5124/5379029845_60f6314172_z.jpg',
    'cats':'https://farm1.staticflickr.com/16/23200321_dcff6ba227_z.jpg',
    'carelephant':'https://farm8.staticflickr.com/7157/6822207699_71e174fd3f_z.jpg',
    'bicycles':'https://farm1.staticflickr.com/33/67728109_0d11a646ef_z.jpg',
    'teddybears':'https://farm4.staticflickr.com/3584/3554604954_2e01e4d007_z.jpg',
}

for key in urlDict.keys():
    rv = requests.get(urlDict[key])
    assert rv.status_code == 200, '画像のダウンロードに失敗しました'
    with io.BytesIO(rv.content) as buf:
        img = Image.open(buf)
        fn = f'{key}.png'
        print(fn)
        fig, ax = plt.subplots(1)
        ax.imshow(img)
        ax.axis('off')
        img.save(fn)

#### 実験3

上記のサンプル画像で実験してみましょう．

In [ ]:
#@markdown 以下で 0 から 4 までの数をひとつ選んでからこのセルを実行すると，物体検出の結果が表示されます．
i = 0 #@param [0, 1, 2, 3, 4] {type: 'raw'}

key = list(urlDict.keys())[i]
fn = f'{key}.png'
print(fn)

# 画像を読み込んでモデルへの入力に加工
img = read_image(fn)
X = torch.unsqueeze(preprocess(img), axis=0).to(device)

# モデルの出力を計算
Y = model(X)[0]

# 出力の情報を画像に加工
labels2 = [weights.meta['categories'][i] for i in Y['labels']]
bbox = draw_bounding_boxes(img, boxes=Y['boxes'], labels=labels2, colors='#00ff00', width=3)
imgResult = to_pil_image(bbox.detach())
imgResult

#### 実験4

自分で用意した画像でも実験してみましょう．

- 81種類の物体クラスのリストを参考に，適当な画像をいろいろ探してみよう
- 面白い／不思議な結果が得られたら takataka に見せてくれると喜びます



In [ ]:
# 以下の 0 を 1 に修正してセルを実行しよう
#
if 1 == 0:
    # Colab へファイルをアップロード
    try:
        from google.colab import files
        rv = files.upload()
    except:
        print('このコードは Colab 以外の環境では実行できないよ．')

In [ ]:
fn = 'teddybears.png' # ここを拡張子含めてアップロードしたファイルの名前に変更
try:
    img = read_image(fn, mode=ImageReadMode.RGB)
    X = torch.unsqueeze(preprocess(img), axis=0)
    Y = model(X)[0]
    labelss = [weights.meta['categories'][i] for i in Y['labels']]
    bbox = draw_bounding_boxes(img, boxes=Y['boxes'], labels=labelss, colors='#00ff00', width=3)
    imgResult = to_pil_image(bbox.detach())
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(imgResult)
    ax.axis('off')
except FileNotFoundError:
    print(f'ファイル {fn} が見つかりません．このセルの1行目を正しいファイル名に変更して実行し直してね．')